## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `Solucao RAG.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [1]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain

In [3]:
# Load dos modelos (Embeddings e LLM)
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=200,
    
)

In [4]:
# Carregar o PDF

pdf_link = "DOC-SF238339076816-20230503.pdf"
loader = PyPDFLoader(pdf_link, extract_images=False)
pages = loader.load_and_split()


In [5]:
# Separar em Chunks (Pedaços de documento)

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True,
)

chunks = text_spliter.split_documents(pages)

In [6]:
# Salvar no Vector DB - Chroma

db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="text_index")
db.persist()


/tmp/ipykernel_94346/2466923040.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [7]:
# Carregar DB

vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

# Load Retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# Chain - Contrução da cadeira de prompt para chamada do LLM
chain = load_qa_chain(llm, chain_type="stuff")


/tmp/ipykernel_94346/1100091879.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)
/tmp/ipykernel_94346/1100091879.py:9: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on

In [17]:
def ask(question):
    context = retriever.get_relevant_documents(question)
    answer = (chain({"input_documents": context, "question": question}, return_only_outputs=True))["output_text"]
    return answer, context


In [18]:
user_question = input("User: ")
answer, context = ask(user_question)
print("Answer: ", answer)

Answer:  Alguns dos principais pontos da Lei de Inteligência Artificial mencionada são:

1. O desenvolvimento, fornecimento ou utilização de sistemas de inteligência artificial de risco excessivo pode resultar em multa e até na suspensão parcial ou total das atividades de uma pessoa jurídica.
2. A obrigação de reparação integral do dano causado, independentemente das sanções aplicadas.
3. A possibilidade de autorização para funcionamento de ambiente regulatório experimental para inovação em inteligência artificial (sandbox regulatório) para entidades que preencham os requisitos.
4. As sanções administrativas serão aplicadas após procedimento que permita a ampla defesa e observando vários critérios, como gravidade da infração, boa-fé do infrator, reincidência, entre outros.
5. A criação e manutenção de uma base de dados de inteligência artificial de


In [19]:
for c in context:
    print(c)

page_content='§ 3º O disposto neste artigo não substitui a aplicação de sanções 
administrativas, civis ou penais definidas na Lei nº 8.078, de 11 de setembro de 
1990, na Lei nº 13.709, de 14 de agosto de 2018, e em legislação específica. 
§ 4º No caso do desenvolvimento, fornecimento ou utilização de 
sistemas de inteligência artificial de risco excessivo haverá, no mínimo, 
aplicação de multa e, no caso de pessoa jurídica, a suspensão parcial ou total, 
provisória ou definitiva de suas atividades. 
§ 5º A aplicação das sanções previstas neste artigo não exclui, em 
qualquer hipótese, a obrigação da reparação integral do dano causado, nos 
termos do art. 27. 
Art. 37. A autoridade competente definirá, por meio de 
regulamento próprio, o procedimento de apuração e critérios de aplicação das 
sanções administrativas a infrações a esta Lei, que serão objeto de consulta 
pública, sem prejuízo das disposições do Decreto-Lei nº 4.657, de 4 de 
setembro de 1942, Lei nº 9.784, de 29 de janei